# Dash applications

This tutorial demonstrates Dash, a Python library for creating web applications containing dashboards made of visualizations, entirely in Python.

Dash uses several libraries under the hood:
*   Flask - the web server.
*   React - UI components.
*   Plotly - data visualization.

Some exercises may require further tools for a better or simple solution. Feel free to check out the official documentation, ask for help, search for a hint, or even ask a generative AI for problem solving. It is also encouraged to copy previous codes for a new task. Starred problems are optional and may require further ideas, ideal for deeper understanding.

**Important note:** Dash apps are hosted on a server, which requires some extra setup and configuration, highly depending on the environment and software versions. This tutorial is designed to work in Google Colab, but please note that the support and the interfaces may change in the future.

# Preparations

It may be necessary to install or upgrade the packages to the newest version, by running the following cell.

In [ ]:
!pip install plotly dash dash-bootstrap-components

Plotly Express is used creating figures. Dash has several components, many of which are imported. Dash Bootrstrap Components are needed for some formatting tasks.

In [ ]:
import plotly.express as px
from dash import Dash, html, dcc, Input, Output, State
import dash_bootstrap_components as dbc

Note: there are some more imports in the notebook, needed for specific tasks.

In this demonstration, the Tips dataset is used, which is available in Plotly (and also in Seaborn). This dataset contains a single table, where each row is a restaurant bill, containing total amount paid, the tip paid, the sex of the person who payed the bill, whether there was a smoker in the group, the day of the week, whether it was dinner or lunch time, and the number of people in the group.

In [ ]:
df = px.data.tips()

df.head()

In this demonstration, the workflow for setting up and running a Dash app is the following.

1.   Creating the app object.
2.   Setting layout.
3.   Registering callback functions.
4.   Running the app.

The creating and running steps may be specific to the environment, so these are encapsulated here in two functions `create_my_app()` and `run_my_app()`, and are used throughout the notebook. These functions work in Google Colab, as of December, 2025. If the notebook does not work in a specific environment, try adjusting these functions first.

In [ ]:
def create_my_app():
  app = Dash(
      __name__,
      external_stylesheets=[dbc.themes.SANDSTONE], # light theme
      # external_stylesheets=[dbc.themes.CYBORG], # dark theme
  )
  return app

from google.colab import output as colab_output
def run_my_app(app):
  colab_output.serve_kernel_port_as_iframe(8050, "/", width=800)
  app.run(host="0.0.0.0", port=8050, debug=False)

It is recommended to re-create the app object each time the server runs.

An alternative method would be to call `app.run()` on a separate thread. Then, the cell does not block execution. However, the app keeps running in the background, so starting a new server on the same port would not work unless the original server is killed. The recommended command to kill the server is the following. Note that it is not needed in the current setup.

In [ ]:
!fuser -k 8050/tcp 2>/dev/null

# Basics

The following app shows a bar plot of tip distribution, further grouped by dining time, on a given day (chosen by a dropdown list), and between a given range of total bill values (chosen by a range slider). If the dropdown list or the changes in value or the button is clicked, then the plot is redrawn according to the updated filters, and the status text is also updated.

In [ ]:
app = create_my_app()

bill_min = df["total_bill"].min()
bill_max = df["total_bill"].max()

app.layout = html.Div([
    html.H1("Tips distribution"),

    html.P(
        id="status_text",
        children=f"Settings: (...)",
    ),

    dcc.Dropdown(
        id="day_filter",
        options=[{"label": v, "value": v}
                 for v in ["Thur", "Fri", "Sat", "Sun"]
        ],
        value="Sun",
    ),

    dcc.RangeSlider(
        id="total_bill_filter",
        min=bill_min,
        max=bill_max,
        step=0.1,
        value=[bill_min, bill_max],
        marks={
            bill_min: str(bill_min),
            bill_max: str(bill_max),
        },
    ),

    html.Button(
        "Update Chart",
        id="update_button",
        n_clicks=0,
    ),

    dcc.Graph(id="plot"),
])

@app.callback(
    Output("plot", "figure"),
    Output("status_text", "children"),
    Input("day_filter", "value"),
    State("total_bill_filter", "value"),
    Input("update_button", "n_clicks"),
)
def update(day, total_bill_limits, n_clicks):
    total_bill_min, total_bill_max = total_bill_limits
    df_filtered = df[
        (df["day"] == day) &
        (df["total_bill"].between(total_bill_min, total_bill_max))
    ]
    fig = px.box(
        df_filtered,
        x="tip",
        color="time",
        range_x=[df["tip"].min(), df["tip"].max()],
        title=f"Tips on {day}day, for bills {total_bill_min} - {total_bill_max}",
    )
    return fig, f"Settings: {day}, [{total_bill_min} - {total_bill_max}]"

run_my_app(app)

The layout of the app, set via `app.layout` defines the dash user interface. It can contain `html` and `dcc` components (not to be confused with `dbc`).
*   `html` contains HTML components, which are wrappers for basic HTML tags. For example, `html.H1` and `html.P` display text. Notably, `html.Div` components can be defined, which may group other components together.
*   `dcc` stands for Dash Core Components, and contains components responsible for interactivity, state changes, or JavaScript logic.

There are many `dcc` component types, but notably for the example:
*   Inputs/Controls, for example `dcc.Dropdown` and `dcc.RangeSlider`, but as an exception, `html.Button` also behaves similarly.
*   Graphics, with `dcc.Graph`.

The behavior of components is managed by any number of registered **callback functions**. The following rules apply.

*   The function must be decorated with the callback method of the app object. In the example: `@app.callback(...)`.
*   Any number of output components can be defined by `Output`, and any number of input components by either `Input` or `State`, by the identifier of the component and the part of it that needs to be updated (for outputs) or needs to be read (for inputs).
*   The number of outputs and their relative order must match the return value of the function.
*   The number of inputs and their relative order must match the arguments of the function, positionally.
*   The callback is triggered when any `Input` changes.
*   Each property of the same component can only be an `Output` in at most one callback function.

Most callbacks are triggered once the page is loaded (e.g. dropdown lists with an initial value). This can be avoided by passing `prevent_initial_call=True` to the `callback()` decorator.

### Exercise (Basics)

Create an app which shows a radio button group (`gcc.RadioItems`), which allows the options "icicle", "sunburst", and "treemap". Display a chart according to the type chosen on the radio buttons, which shows total tips, grouped by day, and then time.

# Layout

For the sake of compactness, the following callback registration procedure is encapsulated into a function, and is used throughout the demonstrations.

In [ ]:
def register_scatter_callback(app):
    @app.callback(
        Output("plot", "figure"),
        Output("total_status", "children"),
        Input("smoker_filter", "value"),
        Input("group_size_filter", "value"),
    )
    def update(smoke_filter_list, group_size_filter_list):
        df_filtered = df[
            (df["smoker"].isin(smoke_filter_list)) &
            (df["size"].isin(group_size_filter_list))
        ]
        fig = px.scatter(
            df_filtered,
            x="total_bill",
            y="tip",
            color="sex",
            trendline="ols",
            trendline_scope="trace",
        )
        return fig, f"Number of bills: {len(df_filtered)}"

In the layout of a Dash app, components are positioned vertically by default. The following example demonstrates this, and serves as a starting point.

In [ ]:
app = create_my_app()

app.layout = html.Div([
    html.H1("Tip totals by day and time"),

    html.Div([
        html.Div([
            html.P("Smoker in group:"),
            dcc.Checklist(
                id="smoker_filter",
                options=[{"label": v, "value": v}
                        for v in ["Yes", "No"]
                ],
                value=["Yes", "No"],
            ),
            html.P("Group size:"),
            dcc.Dropdown(
                id="group_size_filter",
                options=[{"label": v, "value": v}
                        for v in sorted(df["size"].unique())
                ],
                value=sorted(df["size"].unique()),
                multi=True,
            ),
        ]),
        dcc.Graph(id="plot"),
    ]),

    html.P(id="total_status", style={"textAlign": "right"}),
])

register_scatter_callback(app)
run_my_app(app)

A row layout can be obtained in multiple ways, for example:

*   Inline-block layout.
*   Flexbox layout.
*   Bootstrap layout.
*   CSS Grid layout.

The inline-block layout requires to pass a `style=` to `html.Div` components to form a row. Note that the `style=` parameter can be used for other purposes as well. A disadvantage of this approach is that block widths must be manually defined.

In [ ]:
app = create_my_app()

app.layout = html.Div([
    html.H1("Tip totals by day and time"),
    html.H2("Inline-bloc layout"),

    html.Div([
        html.Div([
            html.P("Smoker in group:"),
            dcc.Checklist(
                id="smoker_filter",
                options=[{"label": v, "value": v}
                        for v in ["Yes", "No"]
                ],
                value=["Yes", "No"],
            ),
            html.P("Group size:"),
            dcc.Dropdown(
                id="group_size_filter",
                options=[{"label": v, "value": v}
                        for v in sorted(df["size"].unique())
                ],
                value=sorted(df["size"].unique()),
                multi=True,
            ),
        ], style={
            "width": "25%",
            "display": "inline-block",
            "vertical-align": "top",
            "padding": "10px"
        }),

        html.Div([
            dcc.Graph(id="plot")
        ], style={
            "width": "70%",
            "display": "inline-block",
            "padding": "10px"
        })
    ]),

    html.P(id="total_status", style={"textAlign": "right"}),
])

register_scatter_callback(app)
run_my_app(app)

The Flexbox layout uses modern CSS. The `style=` of both the containing and the children `html.Div` components need to be set (although not only `html.Div` components can accept styling). The container has style `"display" : "flex"`, and the children components have style `"flex": <number>`. The flex numbers in the example below denote proportional growth rates. This approach is modern, automatically responsive, and cleaner.

In [ ]:
Űapp = create_my_app()

app.layout = html.Div([
    html.H1("Tip totals by day and time"),
    html.H2("Flexbox layout"),

    html.Div([
        html.Div([
            html.P("Smoker in group:"),
            dcc.Checklist(
                id="smoker_filter",
                options=[{"label": v, "value": v}
                        for v in ["Yes", "No"]
                ],
                value=["Yes", "No"],
            ),
            html.P("Group size:"),
            dcc.Dropdown(
                id="group_size_filter",
                options=[{"label": v, "value": v}
                        for v in sorted(df["size"].unique())
                ],
                value=sorted(df["size"].unique()),
                multi=True,
            ),
        ], style={"flex": 1, "padding": "10px"}),

        html.Div([
            dcc.Graph(id="plot")
        ], style={"flex": 3, "padding": "10px"}),
    ], style={
        "display": "flex",
        "flex-direction": "row",
        "height": "80vh", # otherwise the chart would be narrow
    }),

    html.P(id="total_status", style={"textAlign": "right"}),
])

register_scatter_callback(app)
run_my_app(app)

The Bootstrap layout can produce the prettiest result. It is responsive, and very clean. The Dash Bootstrap Components are required, which are already imported as `dbc`. Then, `dbc.Row` and `dbc.Col` container components can be added to the layout, which work intuitively.

Widths in the example below denote grid column counts (out of 12).

In [ ]:
app = create_my_app()

app.layout = html.Div([
    html.H1("Tip totals by day and time"),
    html.H2("Bootstrap layout"),

    dbc.Row([
        dbc.Col([
            html.P("Smoker in group:"),
            dcc.Checklist(
                id="smoker_filter",
                options=[{"label": v, "value": v}
                        for v in ["Yes", "No"]
                ],
                value=["Yes", "No"],
            ),
            html.P("Group size:"),
            dcc.Dropdown(
                id="group_size_filter",
                options=[{"label": v, "value": v}
                        for v in sorted(df["size"].unique())
                ],
                value=sorted(df["size"].unique()),
                multi=True,
            ),
        ], width=3),

        dbc.Col([
            dcc.Graph(id="plot")
        ], width=9),
    ], style={
        "height": "80vh", # otherwise the chart would be narrow
    }),

    html.P(id="total_status", style={"textAlign": "right"}),
])

register_scatter_callback(app)
run_my_app(app)

For more advanced use cases, the CSS Grid layout can be used. This is very flexible, so ideal for complex dashboards, while still clean.

In [ ]:
app = create_my_app()

app.layout = html.Div([
    html.H1("Tip totals by day and time"),
    html.H2("CSS Grid layout"),

    html.Div([
        html.Div([
            html.P("Smoker in group:"),
            dcc.Checklist(
                id="smoker_filter",
                options=[{"label": v, "value": v}
                        for v in ["Yes", "No"]
                ],
                value=["Yes", "No"],
            ),
            html.P("Group size:"),
            dcc.Dropdown(
                id="group_size_filter",
                options=[{"label": v, "value": v}
                        for v in sorted(df["size"].unique())
                ],
                value=sorted(df["size"].unique()),
                multi=True,
            ),
        ], style={"padding": "10px"}),

        html.Div([
            dcc.Graph(id="plot"),
        ], style={"padding": "10px"}),
    ], style={
        "display": "grid",
        "gridTemplateColumns": "1fr 3fr",
        "columnGap": "20px",
        "height": "80vh", # otherwise the chart would be narrow
    }),

    html.P(id="total_status", style={"textAlign": "right"}),
])

register_scatter_callback(app)
run_my_app(app)

### Exercise (Layout) 1.

Create an app which shows a pie chart of total bills, grouped by group size, and a list of checkboxes **to the right**, for each day, to filter the pie chart.

### *Exercise (Layout) 2.

Create and app which displays total bills and tips in a stacked bar chart, where days are on the X axis, and the two colors represent the total bills and tips.

Accompany with a text input field, and a button next to it. The expected text input should be applied as a filter to the original data source, as if passed directly to `df.query()`. An empty query input should not filter. Example inputs: `sex=="Female"`, `smoker=="Yes" | time=="Dinner"`.

Hint: the following tools could be useful: `df.melt()`, `px.bar()`, `barmode=stacked`, `dcc.Input`.

# Tabs

The `dcc.Tabs` and `dcc.Tab` components can be used to add tabs to the dashboard. In this way, the dashboard can have multiple pages, as shown below.

In [ ]:
app = create_my_app()

app.layout = html.Div([
    html.H1("Tips distribution"),
    html.H2("(A) Demonstration of tabs"),

    dcc.Tabs(value="tab_1", children=[
        dcc.Tab(label="By Smokers", value="tab_1", children=[
            html.Div([
                dcc.Checklist(
                    id="smoker_filter",
                    options=[{"label": v, "value": v}
                              for v in ["Yes", "No"]
                    ],
                    value=["Yes", "No"],
                    style={"flex": 1},
                ),
                dcc.Graph(id="plot_1", style={"flex": 7},),
            ], style={"display": "flex"})
        ]),
        dcc.Tab(label="By Group Size", value="tab_2", children=[
            html.Div([
                dcc.Checklist(
                    id="group_size_filter",
                    options=[{"label": v, "value": v}
                              for v in sorted(df["size"].unique())
                    ],
                    value=sorted(df["size"].unique()),
                    style={"flex": 1},
                ),
                dcc.Graph(id="plot_2", style={"flex": 7}),
            ], style={"display": "flex"})
        ]),
    ]),
])

@app.callback(
    Output("plot_1", "figure"),
    Input("smoker_filter", "value"),
)
def update_1(smoker_list):
    df_filtered = df[df["smoker"].isin(smoker_list)]
    fig = px.strip(
        df_filtered,
        x="tip",
        range_x=[0, df["tip"].max()],
        color="day",
        category_orders={"day": ["Thur", "Fri", "Sat", "Sun"]},
    )
    return fig

@app.callback(
    Output("plot_2", "figure"),
    Input("group_size_filter", "value"),
)
def update_2(group_size_list):
    df_filtered = df[df["size"].isin(group_size_list)]
    fig = px.strip(
        df_filtered,
        x="tip",
        range_x=[0, df["tip"].max()],
        color="day",
        category_orders={"day": ["Thur", "Fri", "Sat", "Sun"]},
    )
    return fig

run_my_app(app)

The `value=` of `dcc.Tabs` denotes the currently selected tab. This is matched with the `value=` of the individual, children `dcc.Tab` components. The `children=` parameter is used to set the contents.

Dashboard components can also be generated **dynamically**. This is demonstrated by dynamic tabs below.

In [ ]:
app = create_my_app()

tab_specs = [
    {
        "value": "tab_smokers",
        "label": "By Smokers",
        "filter_id": "smoker_filter",
        "plot_id": "plot_1",
        "options": ["Yes", "No"],
        "column": "smoker",
    },
    {
        "value": "tab_groups",
        "label": "By Group Size",
        "filter_id": "group_size_filter",
        "plot_id": "plot_2",
        "options": sorted(df["size"].unique()),
        "column": "size",
    },
]

app.layout = html.Div([
    html.H1("Tips distribution"),
    html.H2("(B) Demonstration of tabs - dynamic generation"),
    dcc.Tabs(id="tabs", value="tab_smokers", children=[
        dcc.Tab(label=spec["label"], value=spec["value"])
        for spec in tab_specs
    ]),
    html.Div(id="tab_content"),
])

@app.callback(
    Output("tab_content", "children"),
    Input("tabs", "value"),
)
def render_tab(tab_value):
    spec = next(s for s in tab_specs if s["value"] == tab_value)
    return html.Div([
        html.Div([
            dcc.Checklist(
                id=spec["filter_id"],
                options=[{"label": v, "value": v} for v in spec["options"]],
                value=list(spec["options"]),
                style={"flex": 1},
            ),
            dcc.Graph(id=spec["plot_id"], style={"flex": 7}),
        ], style={"display": "flex"})
    ])

@app.callback(
    Output("plot_1", "figure"),
    Input("smoker_filter", "value"),
)
def update_1(smokers):
    df_filtered = df[df["smoker"].isin(smokers)]
    return px.strip(
        df_filtered,
        x="tip",
        range_x=[0, df["tip"].max()],
        color="day",
        category_orders={"day": ["Thur", "Fri", "Sat", "Sun"]},
    )

@app.callback(
    Output("plot_2", "figure"),
    Input("group_size_filter", "value"),
)
def update_2(sizes):
    df_filtered = df[df["size"].isin(sizes)]
    return px.strip(
        df_filtered,
        x="tip",
        range_x=[0, df["tip"].max()],
        color="day",
        category_orders={"day": ["Thur", "Fri", "Sat", "Sun"]},
    )

run_my_app(app)

Instead of hard-coding tab contents, they are generated dynamically. Tabs share a logic, but have different specialties, which are stored in `tab_specs`. Instead of setting the contents (`children=`) of each `dcc.Tab`, those remain empty, and a `html.Div` component is inserted afterwards, for the dynamic contents.

Three callback functions are used to generate the contents.
*   `render_tab()` is triggered whenever a tab is selected, and populates the `html.Div` component with the desired contents.
*   `update_1()` is triggered whenever the checkboxes on the smokers page change, and specifically, when it is rendered by `render_tab()`.
*   `update_2()` is triggered the same way as `update_1()`, for the second tab.

The code can be furhter improved so that even the callback is dynamic. This is shown below.

In [ ]:
from dash import MATCH, callback_context

app = create_my_app()

tab_specs = [
    {
        "value": "tab_smokers",
        "label": "By Smokers",
        "options": ["Yes", "No"],
        "column": "smoker",
    },
    {
        "value": "tab_groups",
        "label": "By Group Size",
        "options": sorted(df["size"].unique()),
        "column": "size",
    },
]

app.layout = html.Div([
    html.H1("Tips distribution"),
    html.H2("(C) Demonstration of tabs - fully dynamic generation"),
    dcc.Tabs(
        id="tabs",
        value="tab_smokers",
        children=[
            dcc.Tab(label=s["label"], value=s["value"])
            for s in tab_specs
        ],
    ),
    html.Div(id="tab_content"),
])

@app.callback(
    Output("tab_content", "children"),
    Input("tabs", "value"),
)
def render_tab(tab_value):
    spec = next(s for s in tab_specs if s["value"] == tab_value)
    filter_id = {"type": "filter", "column": spec["column"]}
    plot_id   = {"type": "plot",   "column": spec["column"]}
    return html.Div([
        html.Div([
            dcc.Checklist(
                id=filter_id,
                options=[{"label": v, "value": v} for v in spec["options"]],
                value=list(spec["options"]),
                style={"flex": 1},
            ),
            dcc.Graph(
                id=plot_id,
                style={"flex": 7},
            ),
        ], style={"display": "flex"})
    ])

@app.callback(
    Output({"type": "plot", "column": MATCH}, "figure"),
    Input({"type": "filter", "column": MATCH}, "value"),
)
def update_dynamic(filter_values):
    id_dict = callback_context.inputs_list[0]["id"]
    # id_dict is like {"type": "filter", "column": "smoker"}
    #              or {"type": "filter", "column": "size"}
    column = id_dict["column"]
    df_filtered = df[df[column].isin(filter_values)]
    return px.strip(
        df_filtered,
        x="tip",
        color="day",
        range_x=[0, df["tip"].max()],
        category_orders={"day": ["Thur", "Fri", "Sat", "Sun"]},
    )

run_my_app(app)

This implementation requires `MATCH` and `callback_context` to be imported, and dictionary IDs to be used.

Instead of simple string IDs, dictionary IDs can be used when the setup becomes complex. Then, these dictionary IDs can be matched by patters. The `MATCH` keyword ensures that the `{"type": "filter", "column": MATCH}` ID is matched for any of the columns, so a callback is triggered for each column (`"smoker"` and `"size"`). Moreover, the corresponding output is the tab for the same column, matched by `{"type": "plot", "column": MATCH}`. Finally, `callback_context.inputs_list` is used to extract the matched column. Element `[0]` denotes the first (only) input, which contains the dictionary ID at the `"id"` key.

There are other pattern matching keywords in Dash: `ALL`, and `ALLSMALLER`.

This example demonstrates the strength of dynamic content generation. Applications are much more versatile than managing a tabbed layout.

### *Exercise (Tabs) 1.

Improve the code used for the demonstration of dictionary ID matching and dynamic content generation in the following way. Set a single `column_list = ["sex", "smoker", "time", "size"]`, instead of `tab_specs`, which is hard-coded at the beginning, but can be changed. Generate a tab for each column in the list.

### Exercise (Tabs) 2.

Create an app which has two tabs. The first tab shows total bills on a pie chart, grouped by group size, with any multi-select control that can filter for the time of the day. The second tab shows total bills on a single sunburst chart which groups by group size and then time of the day.

The solution is not required to be dynamically generated.

Hint: if a `dcc.Graph` component is not subject to a callback, then its `figure=` property can be set to a Plotly figure directly.

# Storing data

The `dcc.Store` component can be used to store memory.

The following example uses three tabs. The first tab contains a `dcc.Upload` component that lets the user upload a CSV file. The app then reads the file and converts it into a pandas DataFrame. Its contents are encoded in Base64 and stored in browser memory, on client side, by the `dcc.Store` component. The other two tabs show a heatmap by `px.imshow()` and a scatter matrix by `px.scatter_matrix()` based on all numeric columns of the parsed data frame, read from the `dcc.Store` component.

In [ ]:
import base64
import io
import pandas as pd
from dash import no_update

app = create_my_app()

app.layout = html.Div([
    html.H1("Correlation analyzer"),
    dcc.Tabs(value="tab_1", children=[
        dcc.Tab(label="Upload CSV", value="tab_1", children=[
            dcc.Upload(
                id="upload_csv",
                children="Drag and Drop or Select Files",
                multiple=False,
                style={
                    "width": "300px",
                    "padding": "20px",
                    "border": "1px dashed gray",
                    "textAlign": "center"
                }
            ),
            html.Div(id="preview_table")
        ]),
        dcc.Tab(label="px.imshow()", value="tab_2", children=[
            dcc.Graph(id="plot_2"),
        ]),
        dcc.Tab(label="px.scatter_matrix()", value="tab_3", children=[
            dcc.Graph(id="plot_3"),
        ]),
    ]),
    dcc.Store(id="stored_data", storage_type="memory"),
])

@app.callback(
    Output("stored_data", "data"),
    Input("upload_csv", "contents"),
    # State("upload_csv", "filename"), # possible
)
def load_csv(contents):
    if contents is None:
        return no_update
    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    df_current = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
    df_current = df_current.select_dtypes(include=[int, float])
    return df_current.to_dict("records")

@app.callback(
    Output("preview_table", "children"),
    Output("plot_2", "figure"),
    Output("plot_3", "figure"),
    Input("stored_data", "data")
)
def update_preview(data):
    if data is None:
        return "No data uploaded yet.", None, None
    df_current = pd.DataFrame(data)
    preview = html.Div([
        html.H4("Uploaded Data Preview"),
        html.Pre(df.head().to_string())
    ])
    fig_2 = px.imshow(
        df_current.corr(),
        text_auto=".2f",
    )
    fig_3 = px.scatter_matrix(
        df_current,
        dimensions=df_current.columns,
    )
    return preview, fig_2, fig_3

run_my_app(app)

The `data=` property of `dcc.Store` is used to store data that is reused. `dcc.Store` can only store JSON, therefore, the data frame needs to be converted to a dictionary. This is done by `.to_dict("records")` in the first callback function, which is triggered by an uploaded file.

The second callback is triggered when new data is stored, and the preview at the first tab, and the plots at the other tabs are created. The dataframe is constructed from the stored format.

Both callback functions check whether there is actual data uploaded or stored. If not, then the callback exits without generating content. The `no_update` object ensures that the callback function returns in such a way that the outputs are not updated. This is more elegant than passing dummy values.

The `dcc.Store` also has several `storage_type=` values supported:

*   `"memory"` - Cleared when refreshing page, or navigating away.
*   `"session"` - Persists for session, cleared when browser/tab closes.
*   `"local"` - Persists in local storage of browser.

This technique is very useful to store UI state, pass data between callbacks, and support cache expensive computations.

### Exercise (Storing data) 1.

Revisit the exercise from the Basics section. (You can start from its solution.)

*Create an app which shows a radio button group (`gcc.RadioItems`), which allows the options "icicle", "sunburst", and "treemap". Display a chart according to the type chosen on the radio buttons, which shows total tips, grouped by day, and then time.*

Add a checkbox list (`dcc.Checklist`) to filter the data for group size. Store the aggregated data after the filtering in a `dcc.Store`. The charts should be triggered when the stored data changes.

### *Exercise (Storing data) 2.

Create an app which shows a scatter chart of total bills and tips, which can be filtered by day, using a list of checkboxes, which only triggers when an "Apply" button is clicked. Add an "Undo" button which restores the previous state of the checkboxes. (It is sufficient to work for a depth of one step.)

The following should be kept in mind.

*   The same property of the same component can only be `Output` to **at most one callback** function.
*   It is allowed for a property of the same component to be an `Output` and and `Input` or `State` in the same callback function.
*   Callbacks typically run at page load, so stored data should be initialized.

# Selections in Plotly

Plotly has selection tools for many chart types. Scatter charts are particularly well supported: box and lasso selection is available to select a set of points.

What makes it very useful is that selections can be inputs to callback functions.

The following app updates a stacked bar chart based on the selection in a scatter chart.

In [ ]:
app = create_my_app()

app.layout = html.Div([
    html.H1("Tip sums of selected points"),
    dbc.Row([
        dbc.Col([
            dcc.Graph(
                id="plot_1",
                figure=px.scatter(
                    df,
                    x="total_bill",
                    y="tip",
                )),
        ], width=6),
        dbc.Col([
            dcc.Graph(id="plot_2"),
        ], width=6),
    ])
])

@app.callback(
    Output("plot_2", "figure"),
    Input("plot_1", "selectedData"),
)
def update(selected_data):
    if selected_data is None or selected_data["points"] == []:
        df_filtered = df # no selection -> full data
    else:
        indices = [p["pointIndex"] for p in selected_data["points"]]
        df_filtered = df.iloc[indices]

    df_sum = df_filtered.groupby(["day", "time"], as_index=False)["tip"].sum()
    return px.bar(
        df_sum,
        x="day",
        y="tip",
        color="time",
        category_orders={"day": ["Thur", "Fri", "Sat", "Sun"]},
        barmode="stack",
        title=f"Bar plot of {len(df_filtered)} selected points",
    )

run_my_app(app)

The `"selectionData"` property is updated on the scatter chart, so a callback can be triggered whenever a selection is made.

The selection data itself has a specific structure. First, the indices of the selected points are extracted, and the data frame is filtered to the selection using `.iloc[]`.

### Exercise (Selections in Plotly)

Create an app which supports file upload via a `dcc.Upload` component, and expects the merged form of the Hungarian municipalities dataset (`hungarian_municipalities_merged.csv`). When the dataset is uploaded, display Hungarian municipalities in a map by using `px.scatter_geo()`. Any selection should display the total population and area in separate text boxes.